In [ ]:
# pip install azure-storage-blob opencv-python pandas pillow torch torchvision scikit-learn matplotlib


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ------- -------------------------------- 7.3/39.5 MB 34.9 MB/s eta 0:00:01
   ----------------- ---------------------- 17.0/39.5 MB 39.8 MB/s eta 0:00:01
   ----------------------------- ---------- 28.8/39.5 MB 45.7 MB/s eta 0:00:01
   ---------------------------------------- 39.5/39.5 MB 47.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 11.6/11.6 MB 66.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 77.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 66.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ---------------------------------------- 15.9/15.9 MB 83.7 MB/s eta 0:00:00

   ---------------------

In [23]:
# loads our needed packages
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
# loads our csv that has the merged bounding box data
csv_path = 'combined_defects.csv'
df = pd.read_csv(csv_path)

# sets image folder path
image_folder = 'Images'

# loads image by image id
def load_image(image_id, ext='.png'):
    image_path = os.path.join(image_folder, f"{image_id}{ext}")
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    return Image.open(image_path).convert('RGB')

# crops defect region to that stated in the bounding box
def crop_defects(df):
    crops = []
    for idx, row in df.iterrows():
        image_id = row['image_id']
        x, y, w, h = int(row['BX']), int(row['BY']), int(row['Width']), int(row['Height'])
        
        try:
            img = load_image(image_id)  # Load the full image
            crop = img.crop((x, y, x + w, y + h))  # Crop box: (left, upper, right, lower)
            crops.append({
                'image_id': image_id,
                'bbox': (x, y, w, h),
                'crop': crop
            })
        except Exception as e:
            print(f"Skipping defect at index {idx} in image '{image_id}': {e}")
    
    return crops

# executes cropping
cropped_defects = crop_defects(df)
print(f"Cropped {len(cropped_defects)} defect regions.")

# # save cropped images to disk to double check the crop was correct
# output_folder = 'defect_crops'
# os.makedirs(output_folder, exist_ok=True)

# for i, item in enumerate(cropped_defects):
#     filename = f"{item['image_id']}_defect_{i}.png"
#     path = os.path.join(output_folder, filename)
#     item['crop'].save(path)


Cropped 743 defect regions.


In [15]:
# 1. Setup model: pre-trained ResNet18, remove final classification layer
model = resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # remove last fc layer
model.eval()  # inference mode

# 2. Image preprocessing for ResNet input
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                         std=[0.229, 0.224, 0.225])    # ImageNet stds
])

# 3. Function to get embedding from a crop
def get_embedding(crop_img):
    input_tensor = preprocess(crop_img).unsqueeze(0)  # add batch dim
    with torch.no_grad():
        features = model(input_tensor)
    features = features.squeeze().numpy()  # shape (512,)
    return features

# 4. Apply to all cropped defects
features_list = []
for item in cropped_defects:
    crop_img = item['crop']
    emb = get_embedding(crop_img)
    features_list.append({
        'image_id': item['image_id'],
        'bbox': item['bbox'],
        'embedding': emb
    })

print(f"Extracted embeddings for {len(features_list)} defects.")

X = np.array([item['embedding'] for item in features_list])


Extracted embeddings for 743 defects.


In [19]:
# Set number of clusters (choose based on your domain knowledge or try different values)
n_clusters = 5

# Initialize and fit KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)

# Get cluster labels for each defect
labels = kmeans.labels_

# Add cluster labels back to your features_list for reference
for i, item in enumerate(features_list):
    item['cluster'] = labels[i]

# Print how many defects per cluster
from collections import Counter
print("Defect counts per cluster:")
print(Counter(labels))

Defect counts per cluster:
Counter({np.int32(1): 228, np.int32(0): 183, np.int32(4): 155, np.int32(3): 136, np.int32(2): 41})


In [20]:
output_rows = []
for item in features_list:
    image_id = item['image_id']
    bx, by, w, h = item['bbox']
    cluster = item['cluster']
    output_rows.append({
        'image_id': image_id,
        'BX': bx,
        'BY': by,
        'Width': w,
        'Height': h,
        'Cluster': cluster
    })

df_out = pd.DataFrame(output_rows)
df_out.to_csv('defects_clustered.csv', index=False)
print("Saved clustering results to defects_clustered.csv")

Saved clustering results to defects_clustered.csv


In [27]:
# Folder paths
image_folder = 'Images'
output_folder = 'clustered_images'
os.makedirs(output_folder, exist_ok=True)

# Define distinct colors for clusters (extend as needed)
cluster_colors = [
    'white', 'blue', 'green', 'orange', 'lime',
    'cyan', 'black', 'yellow', 'purple', 'gray'
]

# Group defects by image for easier plotting
from collections import defaultdict
defects_by_image = defaultdict(list)
for item in features_list:
    defects_by_image[item['image_id']].append(item)

# Draw bounding boxes with cluster colors on each image
for image_id, defects in defects_by_image.items():
    img_path = os.path.join(image_folder, f"{image_id}.png")  # Adjust extension if needed
    if not os.path.exists(img_path):
        print(f"Image not found for visualization: {img_path}")
        continue
    
    img = Image.open(img_path).convert('RGB')
    draw = ImageDraw.Draw(img)

    for defect in defects:
        bx, by, w, h = defect['bbox']
        cluster = defect['cluster']
        color = cluster_colors[cluster % len(cluster_colors)]
        # Draw rectangle (outline)
        draw.rectangle([bx, by, bx + w, by + h], outline=color, width=3)
    
    # Save or show the image
    out_path = os.path.join(output_folder, f"{image_id}_clustered.png")
    img.save(out_path)
    print(f"Saved clustered image: {out_path}")


Saved clustered image: clustered_images\1_clustered.png
Saved clustered image: clustered_images\10_clustered.png
Saved clustered image: clustered_images\11_clustered.png
Saved clustered image: clustered_images\12_clustered.png
Saved clustered image: clustered_images\13_clustered.png
Saved clustered image: clustered_images\14_clustered.png
Saved clustered image: clustered_images\15_clustered.png
Saved clustered image: clustered_images\16_clustered.png
Saved clustered image: clustered_images\17_clustered.png
Saved clustered image: clustered_images\18_clustered.png
Saved clustered image: clustered_images\19_clustered.png
Saved clustered image: clustered_images\2_clustered.png
Saved clustered image: clustered_images\20_clustered.png
Saved clustered image: clustered_images\21_clustered.png
Saved clustered image: clustered_images\22_clustered.png
Saved clustered image: clustered_images\23_clustered.png
Saved clustered image: clustered_images\24_clustered.png
Saved clustered image: clustered_